#### Zero-Shot Prompting Tutorial

Zero-shot prompting is crucial in modern AI applications as it enables language models to generalize to new tasks without the need for task-specific training data or fine-tuning. This capability significantly enhances the flexibility and applicability of AI systems, allowing them to adapt to a wide range of scenarios and user needs with minimal setup.

Key Components

1. Understanding Zero-Shot Learning: An introduction to the concept and its importance in AI.
2. Prompt Design Principles: Techniques for crafting effective zero-shot prompts.
3. Task Framing: Methods to frame various tasks for zero-shot performance.
4. LangChain Implementation: Leveraging LangChain for structured zero-shot prompting.

Method Details

The tutorial will cover several methods for implementing zero-shot prompting:

Direct Task Specification: Crafting prompts that clearly define the task without examples.

Role-Based Prompting: Assigning specific roles to the AI to guide its responses.

Format Specification: Providing output format guidelines in the prompt.

Multi-step Reasoning: Breaking down complex tasks into simpler zero-shot steps.

Comparative Analysis: Evaluating different zero-shot prompt structures for the same task.

In [1]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

groq_api_key = "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"

llm = ChatGroq(
     groq_api_key = groq_api_key,
     model = "gemma-7b-it"
)

llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f034dcdbc10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f034dcdaf50>, model_name='gemma-7b-it', groq_api_key=SecretStr('**********'))

In [2]:
def create_chain(prompt_template):
    """
    Create a LangChain chain with the given prompt template.
    
    Args:
        prompt_template (str): The prompt template string.
    
    Returns:
        LLMChain: A LangChain chain object.
    """
    prompt = PromptTemplate.from_template(prompt_template)
    return prompt | llm

1. Direct Task Specification

In this section, we'll explore how to craft prompts that clearly define the task without providing examples. This is the essence of zero-shot prompting.

In [3]:
direct_task_prompt = """Classify the sentiment of the following text as positive, negative, or neutral.
Do not explain your reasoning, just provide the classification.

Text: {text}

Sentiment:"""

direct_task_chain = create_chain(direct_task_prompt)

# Test the direct task specification
texts = [
    "I absolutely loved the movie! The acting was superb.",
    "The weather today is quite typical for this time of year.",
    "I'm disappointed with the service I received at the restaurant."
]

for text in texts:
    result = direct_task_chain.invoke({"text": text}).content
    print(f"Text: {text}")
    print(f"Sentiment: {result}")

Text: I absolutely loved the movie! The acting was superb.
Sentiment: Positive
Text: The weather today is quite typical for this time of year.
Sentiment: Neutral
Text: I'm disappointed with the service I received at the restaurant.
Sentiment: Negative


2. Format Specification

Providing output format guidelines in the prompt can help structure the AI's response in a zero-shot scenario.

In [4]:
format_spec_prompt = """Generate a short news article about {topic}. 
Structure your response in the following format:

Headline: [A catchy headline for the article]

Lead: [A brief introductory paragraph summarizing the key points]

Body: [2-3 short paragraphs providing more details]

Conclusion: [A concluding sentence or call to action]"""

format_spec_chain = create_chain(format_spec_prompt)

# Test the format specification prompting
topic = "The discovery of a new earth-like exoplanet"
result = format_spec_chain.invoke({"topic": topic}).content
print(result)

## **New Hope Emerges: Earth-Like Exoplanet Discovered in habitable zone**

**Lead:** Astronomers have announced the groundbreaking discovery of Kepler-186f, a planet in the habitable zone of a sun-like star, orbiting a distant galaxy. This remarkable exoplanet potentially harbors conditions conducive to life as we know it.

**Body:** This newfound planet, located 1,400 light-years away, boasts a size and mass remarkably close to Earth. Its orbit lies within the habitable zone, the range of distances from a star where conditions are favorable for liquid water to exist on the surface. This crucial factor, coupled with its rocky composition and moderate surface temperature, fuel speculation that Kepler-186f could harbor life.

**Conclusion:** This groundbreaking discovery reignites the debate on the prevalence of life beyond our solar system, offering hope that we might soon find concrete evidence of other intelligent life in the cosmos.


3. Multi-step Reasoning

For complex tasks, we can break them down into simpler zero-shot steps. This approach can improve the overall performance of the model.

In [5]:
multi_step_prompt = """Analyze the following text for its main argument, supporting evidence, and potential counterarguments. 
Provide your analysis in the following steps:

1. Main Argument: Identify and state the primary claim or thesis.
2. Supporting Evidence: List the key points or evidence used to support the main argument.
3. Potential Counterarguments: Suggest possible objections or alternative viewpoints to the main argument.

Text: {text}

Analysis:"""

multi_step_chain = create_chain(multi_step_prompt)

# Test the multi-step reasoning approach
text = """While electric vehicles are often touted as a solution to climate change, their environmental impact is not as straightforward as it seems. 
The production of batteries for electric cars requires significant mining operations, which can lead to habitat destruction and water pollution. 
Moreover, if the electricity used to charge these vehicles comes from fossil fuel sources, the overall carbon footprint may not be significantly reduced. 
However, as renewable energy sources become more prevalent and battery technology improves, electric vehicles could indeed play a crucial role in combating climate change."""

result = multi_step_chain.invoke({"text": text}).content
print(result)

## Analysis of the Text:

**1. Main Argument:**

The text argues that while electric vehicles are often presented as a solution to climate change, their environmental impact is complex and requires careful consideration.

**2. Supporting Evidence:**

- Production of EV batteries involves significant mining, leading to habitat destruction and water pollution.
- If charged using fossil fuels, EVs may not significantly reduce carbon footprint.
- As renewable energy and battery technology improve, EVs could eventually become a viable climate change solution.


**3. Potential Counterarguments:**

- **Focus on lifecycle analysis:** The text briefly mentions the need for a lifecycle analysis to assess the complete environmental impact of EVs, including potential benefits in areas like air quality and energy security.
- **Technological advancement:** The text acknowledges the potential for future innovations in battery technology and renewable energy to address the environmental concerns assoc

4. Comparative Analysis

Let's compare different zero-shot prompt structures for the same task to evaluate their effectiveness.

In [6]:
def compare_prompts(task, prompt_templates):
    """
    Compare different prompt templates for the same task.
    
    Args:
        task (str): The task description or input.
        prompt_templates (dict): A dictionary of prompt templates with their names as keys.
    """
    print(f"Task: {task}\n")
    for name, template in prompt_templates.items():
        chain = create_chain(template)
        result = chain.invoke({"task": task}).content
        print(f"{name} Prompt Result:")
        print(result)
        print("\n" + "-"*50 + "\n")

task = "Explain conciesly the concept of blockchain technology"

prompt_templates = {
    "Basic": "Explain {task}.",
    "Structured": """Explain {task} by addressing the following points:
1. Definition
2. Key features
3. Real-world applications
4. Potential impact on industries"""
}

compare_prompts(task, prompt_templates)

Task: Explain conciesly the concept of blockchain technology

Basic Prompt Result:
**Blockchain Technology:**

- A decentralized and distributed digital ledger that records transactions and tracks assets in a secure and transparent manner.
- Consists of blocks linked together in chronological order, forming an immutable chain.
- Transactions are validated by a network of computers (nodes) through consensus mechanism, ensuring trust and security.


**Key Features:**

- **Decentralization:** No single entity controls the blockchain.
- **Transparency:** Transactions are publicly viewable on the blockchain.
- **Security:** Advanced cryptography protects the blockchain from tampering.
- **Immutability:** Once data is recorded on the blockchain, it cannot be altered or deleted.


**How it Works:**

1. Transactions are initiated on the blockchain network.
2. Nodes in the network verify the transaction and add it to a block.
3. The block is then added to the blockchain chain.
4. The consensus 